In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.display import display

from PIL import Image, ImageDraw
import cv2
import numpy as np

In [ ]:
dartboard_filename = 'dart_board.png'

In [ ]:
dartboard_image = Image.open(dartboard_filename)
dartboard_image

In [ ]:
width, height = dartboard_image.size
print(width, height)
left = 450
top = 350
right = width - 480
bottom = height
cropped = dartboard_image.crop((left, top, right, bottom))
cropped.save('dart_board_cropped.png')
cropped

In [ ]:
# draw x on image at position (x, y) with thickness t, color c, and length l
def draw_x_at(img, pos, t=2, c='blue', l=10):
    x, y = pos
    copy = img.copy()
    draw = ImageDraw.Draw(copy)
    draw.line((x-l, y-l, x+l, y+l), fill=c, width=t)
    draw.line((x-l, y+l, x+l, y-l), fill=c, width=t)
    return copy

In [ ]:
cropped_with_anchors = cropped.copy()

top_anchor = (495, 195)
left_anchor = (232, 425)
right_anchor = (750, 442)
bottom_anchor = (487, 643)

cropped_with_anchors = draw_x_at(cropped_with_anchors, top_anchor)
cropped_with_anchors = draw_x_at(cropped_with_anchors, left_anchor)
cropped_with_anchors = draw_x_at(cropped_with_anchors, right_anchor)
cropped_with_anchors = draw_x_at(cropped_with_anchors, bottom_anchor)

display(cropped_with_anchors)

In [ ]:
# draw green x at dart top
def pos_to_lines(pos, line_length):
    x, y = pos
    return [
        ((x - line_length / 2, y - line_length / 2), (x + line_length / 2, y + line_length / 2)),
        ((x + line_length / 2, y - line_length / 2), (x - line_length / 2, y + line_length / 2))
    ]

def draw_x_at(img, pos, t=2, color='blue', length=10):
    x, y = pos
    copy = img.copy()
    draw = ImageDraw.Draw(copy)
    for line in pos_to_lines(pos, length):
        draw.line(line, fill=color, width=t)
    return copy

In [ ]:
###
### Load dart images
###

dart_1 = Image.open('dart_1.png')

line_length = 10
line_width = 1
pos_1 = (111, 19)

line_1, line_2 = pos_to_lines(pos_1, line_length)

dart_1_draw = ImageDraw.Draw(dart_1)
dart_1_draw.line(line_1, fill='green', width=line_width)
dart_1_draw.line(line_2, fill='green', width=line_width)
dart_1.convert('RGB')

In [ ]:
dart_2 = Image.open('dart_2.png')

pos_2 = (97, 39)
line_1, line_2 = pos_to_lines(pos_2, line_length)

dart_2_draw = ImageDraw.Draw(dart_2)
dart_2_draw.line(line_1, fill='green', width=line_width)
dart_2_draw.line(line_2, fill='green', width=line_width)

dart_2.convert('RGB')

In [ ]:
dart_3 = Image.open('dart_3.png')

pos_3 = (40, 28)
line_1, line_2 = pos_to_lines(pos_3, line_length)

dart_3_draw = ImageDraw.Draw(dart_3)
dart_3_draw.line(line_1, fill='green', width=line_width)
dart_3_draw.line(line_2, fill='green', width=line_width)

dart_3.convert('RGB')

In [ ]:
def composite_at(background, foreground, pos):
    if background.mode != 'RGBA':
        background = background.convert('RGBA')

    tmp_img = Image.new('RGBA', background.size, (0, 0, 0, 0))
    tmp_img.paste(foreground, pos)
    return Image.alpha_composite(background, tmp_img)

In [ ]:
composite = composite_at(cropped, dart_1, (300, 200))
composite.convert('RGB')

In [ ]:
def dart_position_in_cropped(pasted_at_pos, dart_top_pos):
    return (pasted_at_pos[0] + dart_top_pos[0], pasted_at_pos[1] + dart_top_pos[1])

def dart_position_in_original(cropped_at_pos, pasted_at_pos, dart_top_pos):
    return (cropped_at_pos[0] + pasted_at_pos[0] + dart_top_pos[0], cropped_at_pos[1] + pasted_at_pos[1] + dart_top_pos[1])

In [ ]:
###
### generate data
###
import random
import json

def random_positions(start_pos, end_pos):
    positions = []
    for i in range(3):
        x = random.randint(start_pos[0], end_pos[0])
        y = random.randint(start_pos[1], end_pos[1])
        positions.append((x, y))
    return positions

def cluster_positions_at_double():
    pass
def cluster_positions_at_triple():
    pass

def build_label(filename, annotations, series_id):
    out_annotations = []
    for bbox, label in annotations:
        out_annotations.append({
            'bbox': bbox,
            'label': label
        })
    
    return {
        'image_filename': filename,
        'series_id': series_id,
        'annotations': out_annotations
    }

save_location = 'dartnet_data'


num_images = 1000
start_pos = (20, 20)
end_pos = cropped.size[0] - 20, cropped.size[1] - 20


orig_dart_1 = Image.open('dart_1.png')
orig_dart_2 = Image.open('dart_2.png')
orig_dart_3 = Image.open('dart_3.png')

dart_images = [orig_dart_1, orig_dart_2, orig_dart_3]
dart_top_positions = [pos_1, pos_2, pos_3]
print(f'dart top pos: {dart_top_positions}')

print(f'cropped size {cropped.size}')

anchor_positions = [left_anchor, top_anchor, right_anchor, bottom_anchor]
anchor_annotations = [(a, 'anchor') for a in anchor_positions]

dart_max_width = max([x.width for x in dart_images])
dart_max_height = max([x.height for x in dart_images])
end_pos = end_pos[0] - dart_max_width, end_pos[1] - dart_max_height
print(f'start pos {start_pos}')
print(f'end pos {end_pos}')


labels = []
for i in range(num_images):
    print(f'iteration: {i}')
    img_pos_1, img_pos_2, img_pos_3 = random_positions(start_pos, end_pos)

    dart_idx = random.randint(0, len(dart_images) - 1)
    dart_1 = dart_images[dart_idx]
    dart_1_top_pos = dart_top_positions[dart_idx]
    composite_1 = composite_at(cropped, dart_1, img_pos_1)

    dart_idx = random.randint(0, len(dart_images) - 1)
    dart_2 = dart_images[dart_idx]
    dart_2_top_pos = dart_top_positions[dart_idx]
    composite_2 = composite_at(composite_1, dart_2, img_pos_2)

    dart_idx = random.randint(0, len(dart_images) - 1)
    dart_3 = dart_images[dart_idx]
    dart_3_top_pos = dart_top_positions[dart_idx]
    composite_3 = composite_at(composite_2, dart_3, img_pos_3)
    
    dart_1_pos_in_cropped = dart_position_in_cropped(img_pos_1, dart_1_top_pos)
    dart_2_pos_in_cropped = dart_position_in_cropped(img_pos_2, dart_2_top_pos)
    dart_3_pos_in_cropped = dart_position_in_cropped(img_pos_3, dart_3_top_pos)

    print(f'image_1_pos: {img_pos_1}, image_2_pos: {img_pos_2}, image_3_pos: {img_pos_3}')
    
    # save images as pngs
    filename_1 = f'{save_location}/dart_{i}_1.png'
    filename_2 = f'{save_location}/dart_{i}_2.png'
    filename_3 = f'{save_location}/dart_{i}_3.png'
    
    
    composite_1.save(filename_1)
    composite_2.save(filename_2)
    composite_3.save(filename_3)
    
    combinations = [
        [filename_1, [(dart_1_pos_in_cropped, 'dart'), *anchor_annotations]],
        [filename_2, [(dart_1_pos_in_cropped, 'dart'), (dart_2_pos_in_cropped, 'dart'), *anchor_annotations]],
        [filename_3, [(dart_1_pos_in_cropped, 'dart'), (dart_2_pos_in_cropped, 'dart'), (dart_3_pos_in_cropped, 'dart'), *anchor_annotations]],
    ]
    l1 = build_label(*combinations[0], i)
    l2 = build_label(*combinations[1], i)
    l3 = build_label(*combinations[2], i)
    
    
    labels += [l1, l2, l3]

with open(f'{save_location}/labels.json', 'w') as json_file:
    json_str = json.dumps({'labels': labels}, indent=4)
    print(json_str)
    json_file.write(json_str)


l = labels[2]
img = Image.open(l['image_filename'])
for a in l['annotations']:
    bbox = a['bbox']
    print(f'draw bbox {bbox}')
    img = draw_x_at(img, (bbox[0], bbox[1]), length=10)

img.convert('RGB')